# 8: Creating a NLP model that can predict which expression is better between two sentences

It is supposed to run on google colab

In [ ]:
! pip install datasets transformers
! pip install --upgrade pandas
! apt install git-lfs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import transformers
import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric

4.25.1


# Fine-tuning a model on a text classification task

## Loading the dataset

In [ ]:
metric = load_metric('glue', 'wnli')

<ipython-input-7-8fefc5f03c32>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', 'wnli')


In [ ]:
import pickle
import pandas as pd
df = pickle.load(open('/content/drive/MyDrive/BCUFR/dataset_for_segmentation.p', 'rb'))
df.head()

,idx,filename,caption,model,expr,conf,bbox,labels,image_id,caption_raw,caption_preprocessed,title_raw,title_preprocessed
0,1027,JOMU_32980_2k_324w.jpg,people buying sweets at the market.,GLIP,caption,"[tensor(0.8280), tensor(0.7924), tensor(0.6899...","[[tensor(178.8594), tensor(116.5122), tensor(2...","[people, people, people, people, people, peopl...",1488.0,people buying sweets at the market.,people buying sweets at the market.,"Saint Nicholas Festival Market, Place de Notre...","saint nicholas festival market, place de notre..."
1,183,CAPO_02480_2k_324w.jpg,a group of soldiers stand in front of a building.,GLIP,caption,"[tensor(0.8242), tensor(0.5884), tensor(0.5781...","[[tensor(0.8775), tensor(131.9302), tensor(81....","[soldiers, a building, a building, a building]",516.0,a group of soldiers stand in front of a building.,a group of soldiers stand in front of a building.,[Mobilization at Perolles in August 1914],[mobilization at perolles in august 1914]
2,543,JATH_26232_2k_324w.jpg,soldiers stand in front of a military vehicle.,GLIP,caption,"[tensor(0.7898), tensor(0.7305), tensor(0.6872...","[[tensor(97.0861), tensor(137.5495), tensor(14...","[soldiers, soldiers, a military vehicle, soldi...",950.0,soldiers stand in front of a military vehicle.,soldiers stand in front of a military vehicle.,"Additional service for women, Barracks de la P...","additional service for women, barracks de la p..."
3,430,JATH_10616_2k_324w.jpg,women walking down a street.,GLIP,caption,"[tensor(0.6900), tensor(0.6777), tensor(0.6676...","[[tensor(125.4207), tensor(111.6184), tensor(1...","[women, women, women, a street, women]",836.0,women walking down a street.,women walking down a street.,Procession on the route to the Alps during a w...,procession on the route to the alps during a w...
4,408,HAWI_01023_2k_324w.jpg,a photograph of a large tropical cyclone.,GLIP,caption,[tensor(0.7263)],"[[tensor(4.5861), tensor(13.1419), tensor(318....",[a large tropical cyclone],814.0,a photograph of a large tropical cyclone.,a a large tropical cyclone.,Tornado over Lake Morat,tornado over lake morat


In [ ]:
import pandas
import torch

sep_token = "[SEP]"


df_temp = pd.DataFrame(columns=['label', 'caption', 'title'])
df_temp['caption'] = df.caption_raw.str.lower()
df_temp['title'] = df.title_raw.str.lower()
df_temp['label'] = df.caption == df.title_raw
df_temp['label'] = df_temp.label.astype(int)
df_temp['title'] = df_temp['title'].astype(str)
df_temp['caption'] = df_temp['caption'].astype(str)
df_temp = df_temp.reset_index().rename({'index':'idx'}, axis=1)
df_temp.head()

,idx,label,caption,title
0,0,0,people buying sweets at the market.,"saint nicholas festival market, place de notre..."
1,1,0,a group of soldiers stand in front of a building.,[mobilization at perolles in august 1914]
2,2,0,soldiers stand in front of a military vehicle.,"additional service for women, barracks de la p..."
3,3,0,women walking down a street.,procession on the route to the alps during a w...
4,4,0,a photograph of a large tropical cyclone.,tornado over lake morat


In [ ]:
# swap caption and title for 50% of the data, switch label for them
df_temp2 = df_temp.copy()
df_temp2['caption'] = df_temp['title']
df_temp2['title'] = df_temp['caption']
df_temp2['label'] = df_temp2['label'].apply(lambda x: 1-x)
df_temp2.head()

df_temp = pd.concat([df_temp, df_temp2])
df_temp['title'] = df_temp.title.replace('[','').replace(']','')
df_temp['caption'] = df_temp.caption.replace('[','').replace(']','')
df_temp['sentence'] = df_temp.caption + sep_token +  df_temp.title
df_temp = df_temp.reset_index().rename({'index':'idx'}, axis=1)


To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
from datasets import Dataset
dataset = Dataset.from_dict(df_temp)
dataset = dataset.train_test_split(test_size=0.3)

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset['train'])

,idx,label,caption,title,sentence
0,902,1,"brasserie du cardinal, fribourg",a group of workers in a factory.,"brasserie du cardinal, fribourg[SEP]a group of workers in a factory."
1,386,1,"rossens dam, construction, young worker",actor in a promotional photo for the film.,"rossens dam, construction, young worker[SEP]actor in a promotional photo for the film."
2,336,0,physicist at work in the laboratory.,unda assembly at the university of fribourg,physicist at work in the laboratory.[SEP]unda assembly at the university of fribourg
3,789,1,"advertising photo of clothing, fribourg",martial artist poses for a portrait.,"advertising photo of clothing, fribourg[SEP]martial artist poses for a portrait."
4,1339,1,"denis honegger, architect of the building of the university of fribourg, site miséricorde","portrait of actor wearing a suit and tie, 1950s.","denis honegger, architect of the building of the university of fribourg, site miséricorde[SEP]portrait of actor wearing a suit and tie, 1950s."
5,34,1,"cortège de la saint-nicolas, crowd, fribourg",a group of children dressed as soldiers stand in front of a building.,"cortège de la saint-nicolas, crowd, fribourg[SEP]a group of children dressed as soldiers stand in front of a building."
6,1362,1,dandelion,dandelion seeds blowing in the wind.,dandelion[SEP]dandelion seeds blowing in the wind.
7,1426,1,"la motta baths, school day swimming events (kinderfest), fribourg",a swimmer jumps into the water.,"la motta baths, school day swimming events (kinderfest), fribourg[SEP]a swimmer jumps into the water."
8,1161,1,"sunbed on the prealps, glâne district",sunrise over a lake in the countryside.,"sunbed on the prealps, glâne district[SEP]sunrise over a lake in the countryside."
9,828,0,photograph of a rocket being launched from a building.,graf-zeppelin above morat,photograph of a rocket being launched from a building.[SEP]graf-zeppelin above morat


## Preprocessing the data

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)

In [ ]:

def preprocess_function(examples):
    return tokenizer(
    examples['sentence'] ,  # The string to encode
    add_special_tokens=True,  # Add the special tokens (e.g. [CLS], [SEP])
    truncation=True,  # Truncate sequences that are too long
)

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [ ]:
metric_name = "accuracy"
model_name = 'Heritage-in-the-Digital-Age'

args = TrainingArguments(
    f"{model_name}-expression-comparison",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=15,
    per_device_eval_batch_size=15,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return metric.compute(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
validation_key = "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: title, sentence, caption, idx. If title, sentence, caption, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 873
  Batch size = 15


{'eval_loss': 0.5280570387840271,
 'eval_accuracy': 0.7709049255441008,
 'eval_runtime': 1.3634,
 'eval_samples_per_second': 640.291,
 'eval_steps_per_second': 43.273,
 'epoch': 5.0}

To see how your model fared you can compare it to the [GLUE Benchmark leaderboard](https://gluebenchmark.com/leaderboard).

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to Heritage-in-the-Digital-Age-expression-comparison
Configuration saved in Heritage-in-the-Digital-Age-expression-comparison/config.json
Model weights saved in Heritage-in-the-Digital-Age-expression-comparison/pytorch_model.bin
tokenizer config file saved in Heritage-in-the-Digital-Age-expression-comparison/tokenizer_config.json
Special tokens file saved in Heritage-in-the-Digital-Age-expression-comparison/special_tokens_map.json
To https://huggingface.co/tgieruc/Heritage-in-the-Digital-Age-expression-comparison
   87c36a5..3791aa5  main -> main

   87c36a5..3791aa5  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.7709049255441008}]}


'https://huggingface.co/tgieruc/Heritage-in-the-Digital-Age-expression-comparison/commit/3791aa54cd2454360db571fd04486cfe5b24924e'